In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#Extracting the information from the web page.
res = requests.get("https://en.wikipedia.org/wiki/List_of_cryptocurrencies")
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
datos=soup.find_all('table')[0]
datos.find_all('tr')[1].find_all("td")[0]

<td>2009
</td>

In [4]:
def createDF(table):
    headers = [header.text[:-1] for header in datos.find_all('th')]
    data=[]
    for row in table.find_all('tr')[1:]:
        currency = {}
        for index,value in enumerate(row.find_all("td")):
            currency[headers[index]]=value.text[:-1]
        data.append(currency)
    data = pd.DataFrame(data)
    return data

In [5]:
wiki=createDF(datos)
wiki

,Release,Currency,Symbol,Founder(s),Hash algorithm,Programming language of implementation,"Cryptocurrency blockchain (PoS, PoW, or other)",Notes
0,2009,Bitcoin,"BTC,[4] XBT, ₿",Satoshi Nakamoto[nt 1],SHA-256d[5][6],C++[7],PoW[6][8],The first and most widely used decentralized l...
1,2011,Litecoin,"LTC, Ł",Charlie Lee,Scrypt,C++[11],PoW,One of the first cryptocurrencies to use Scryp...
2,2011,Namecoin,NMC,Vincent Durham[12][13],SHA-256d,C++[14],PoW,"Also acts as an alternative, decentralized DNS."
3,2012,Peercoin,PPC,Sunny King(pseudonym)[citation needed],SHA-256d[citation needed],C++[15],PoW & PoS,The first cryptocurrency to use POW and POS fu...
4,2013,Dogecoin,"DOGE, XDG, Ð",Jackson Palmer& Billy Markus[16],Scrypt[17],C++[18],PoW,Based on the Doge internet meme.
5,2013[citation needed],Gridcoin,GRC,Rob Hälford[citation needed],Scrypt,C++[19],Decentralized PoS,Linked to citizen science through the Berkeley...
6,2013,Primecoin,XPM,Sunny King(pseudonym)[citation needed],1CC/2CC/TWN[21],"TypeScript, C++[22]",PoW[21],Uses the finding of prime chains composed of C...
7,2013,Ripple[23][24],XRP,Chris Larsen &Jed McCaleb[25],ECDSA[26],C++[27],"""Consensus""",Designed for peer to peer debt transfer. Not b...
8,2013,Nxt,NXT,BCNext(pseudonym),SHA-256d[28],Java[29],PoS,Specifically designed as a flexible platform t...
9,2014,Auroracoin,AUR,Baldur Odinsson(pseudonym)[30],Scrypt,C++[31],PoW,Created as an alternative currency for Iceland...


In [6]:
def quitarBrackets():
    for columna in wiki:
        wiki[columna] = wiki[columna].str.replace(r"\[.*\]","")
    return wiki
clean_wiki = quitarBrackets()